In [6]:
import numpy as np
import pandas as pd

In [7]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil #for file operaton like create copy etc
from tqdm import tqdm #it show loop or itration as bars
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [8]:
SEED = 18
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [9]:
data_root = 'C:\shyam\A multiclass disease classifier\data\chest_xray'
print(os.listdir(data_root))
train_dir = 'train'
val_dir = 'val'
each_val_img_ind = 50
#every 50 image from train data goes to validation set for further model validation 


['test', 'train']


<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Shyam Kadiwar\AppData\Local\Temp\ipykernel_14340\608491056.py:1: SyntaxWarning: invalid escape sequence '\s'
  data_root = 'C:\shyam\A multiclass disease classifier\data\chest_xray'


In [14]:
class_names = ['NORMAL', 'PNEUMONIA']

In [15]:
#making or creating file for train and validation
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)
    os.makedirs(os.path.join(dir_name, 'virus'), exist_ok=True)

In [16]:
#copying image to respective dir like val to val and train to train

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if 'virus' in file_name:
            dest_dir = os.path.join(train_dir, 'virus') if i % each_val_img_ind != 0 else os.path.join(val_dir, 'virus')
        else:
            dest_dir = os.path.join(train_dir, class_name) if i % each_val_img_ind != 0 else os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))
class_names.append('virus')

100%|██████████| 3875/3875 [01:45<00:00, 36.60it/s]


In [17]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
#we do augmentation because model can learn on various transformation 

In [18]:
val_datagen = ImageDataGenerator(rescale=1./255) #each pixle is divide into 255

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=10,
    class_mode='categorical'
)

Found 5111 images belonging to 3 classes.
Found 105 images belonging to 3 classes.


In [19]:
#defining the model with 8 convulation layers and 4 maxpooling layers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(len(class_names), activation='softmax'))

model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
              loss=losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

c:\Users\Shyam Kadiwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
#training the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10


c:\Users\Shyam Kadiwar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


512/512 ━━━━━━━━━━━━━━━━━━━━ 1890s 4s/step - accuracy: 0.4698 - loss: 1.0920 - val_accuracy: 0.4190 - val_loss: 1.1211
Epoch 2/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 608s 1s/step - accuracy: 0.4877 - loss: 1.0505 - val_accuracy: 0.4190 - val_loss: 1.0898
Epoch 3/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 604s 1s/step - accuracy: 0.5005 - loss: 1.0413 - val_accuracy: 0.4190 - val_loss: 1.0855
Epoch 4/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 620s 1s/step - accuracy: 0.4994 - loss: 1.0426 - val_accuracy: 0.4190 - val_loss: 1.0848
Epoch 5/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 607s 1s/step - accuracy: 0.4804 - loss: 1.0542 - val_accuracy: 0.4190 - val_loss: 1.1059
Epoch 6/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 611s 1s/step - accuracy: 0.4770 - loss: 1.0563 - val_accuracy: 0.4190 - val_loss: 1.0926
Epoch 7/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 607s 1s/step - accuracy: 0.5038 - loss: 1.0379 - val_accuracy: 0.4190 - val_loss: 1.0877
Epoch 8/10
512/512 ━━━━━━━━━━━━━━━━━━━━ 587s 1s/step - accuracy: 0.4773 - loss: 1.0552 - val_accuracy: 0.41

In [22]:
model.save('pnemonia_model.h5')